In [75]:
root = "C:/Users/egorg/Projects/MemcoinsTradingBotPolygon"; import sys; sys.path.append(root)

In [ ]:
import numpy as np
import src.utils as util
import matplotlib.pyplot as plt

from src.env.data import Dataset, ParametersSet

In [77]:
data_path = f"{root}/data"
shift = int(180 / 60)
datasets = util.load_datasets(data_path, shift)
len(datasets)

5049

In [78]:
df = util.dataset_to_dataframe(datasets)
df = df[df["trial_rank"] == 0]
df

,dataset,std,rstd,ma_distance_ratio,rsi_crossovers,rsi_crossunders,entropy,regression_slope,regression_sqr,volatility_clustering,...,drawdown_convexity,drop_velocity,regime,trial_rank,profit,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0_10,6.523627e-08,0.136456,0.048904,1,0,3.023363,0.002367,0.784891,0.045169,...,0.000027,0.711631,23,0,1005.423851,5,0.0460,0.03290,0.0491,0.00751
20,0_11,3.956624e-08,0.080538,0.027044,1,0,3.189496,-0.001863,0.944745,-0.086710,...,-0.000034,0.562611,25,0,1204.695376,5,0.1185,0.03895,0.0734,0.03944
40,0_12,2.014306e-08,0.044340,0.019405,1,0,2.179471,-0.000821,0.972760,-0.026485,...,0.000006,0.347130,1,0,1307.900000,3,0.0522,0.02336,0.0469,0.02719
60,0_13,2.400482e-08,0.056238,0.017800,1,0,2.219783,-0.001350,0.980654,-0.048117,...,-0.000014,0.271950,1,0,1365.978294,4,0.0509,0.02897,0.0327,0.02457
80,0_14,1.724124e-08,0.042506,0.024766,1,0,2.412590,-0.000219,0.966406,0.429391,...,-0.000012,2.363053,12,0,1144.787500,6,0.0912,0.07686,0.0964,0.08304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100880,9_4,2.109971e-07,0.085310,0.021940,4,0,2.649813,-0.002241,0.866363,0.056508,...,-0.000018,0.431340,25,0,1314.253231,6,0.0425,0.01345,0.0374,0.03294
100900,9_5,1.526912e-07,0.067886,0.025938,4,0,1.965845,-0.001244,0.564301,0.122511,...,0.000015,0.268158,36,0,1040.690137,2,0.0491,0.01573,0.0721,0.00931
100920,9_6,2.925304e-07,0.120796,0.043184,2,1,1.646462,0.002239,0.470154,-0.016312,...,-0.000017,0.775788,20,0,1142.515861,4,0.0357,0.01171,0.0803,0.01613
100940,9_7,1.469328e-07,0.054975,0.034891,0,1,1.963904,0.000559,0.368129,-0.018669,...,-0.000016,0.204268,38,0,1207.897028,4,0.0310,0.01410,0.0689,0.04989


In [79]:
eps = 1e-12
def normalize_path(x: np.ndarray, target_len: int) -> np.ndarray:
    x = x.astype(np.float64)
    if len(x) != target_len:
        t_old = np.linspace(0, 1, len(x))
        t_new = np.linspace(0, 1, target_len)
        x = np.interp(t_new, t_old, x)
    x = x - np.mean(x)
    std = np.std(x)
    if std < eps: return np.zeros_like(x)
    return x / std

In [80]:
def similiarity_score(a: Dataset, b: Dataset) -> np.float64:
    ma_a = a.features.graphical.ma15
    ma_b = b.features.graphical.ma15
    L = min(len(ma_a), len(ma_b))
    if L < 10:
        return np.float64(0.0)
    na = normalize_path(ma_a, L)
    nb = normalize_path(ma_b, L)
    dist = np.sqrt(np.mean((na - nb) ** 2))
    sim = np.exp(-dist)
    return np.float64(sim)

def simple_similiarity_score(a: Dataset, b: Dataset) -> np.float64:
    ma_a = a.features.graphical.ma15.astype(np.float64)
    ma_b = b.features.graphical.ma15.astype(np.float64)
    L = min(len(ma_a), len(ma_b))
    if L < 3:
        return np.float64(0.0)
    ma_a = ma_a[:L]
    ma_b = ma_b[:L]
    pct_a = np.diff(ma_a) / (ma_a[:-1] + eps)
    pct_b = np.diff(ma_b) / (ma_b[:-1] + eps)
    std_a = np.std(pct_a)
    std_b = np.std(pct_b)
    if std_a > eps:
        pct_a = pct_a / std_a
    if std_b > eps:
        pct_b = pct_b / std_b
    diff = np.abs(pct_a - pct_b)
    step_sim = 1.0 / (1.0 + diff)
    return np.float64(np.mean(step_sim))

In [ ]:
groups: list[list[Dataset]] = []
computed_indexes: list[int] = []
for this_index, this in enumerate(datasets):
    if this_index in computed_indexes: continue
    computed_indexes.append(this_index)
    buf: list[Dataset] = [this]
    for other_index, other in enumerate(datasets):
        if other_index in computed_indexes or this.name == other.name: continue
        score = simple_similiarity_score(this, other)
        if score >= 0.6:
            computed_indexes.append(other_index)
            buf.append(other)
    groups.append(buf)

In [ ]:
dataset_to_regime: dict[str, int] = {}
for regime, group in enumerate(groups):
    for ds in group:
        dataset_to_regime[ds.name] = regime
df["regime"] = df["dataset"].map(dataset_to_regime)

In [86]:
df

,dataset,std,rstd,ma_distance_ratio,rsi_crossovers,rsi_crossunders,entropy,regression_slope,regression_sqr,volatility_clustering,...,drawdown_convexity,drop_velocity,regime,trial_rank,profit,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0_10,6.523627e-08,0.136456,0.048904,1,0,3.023363,0.002367,0.784891,0.045169,...,0.000027,0.711631,0,0,1005.423851,5,0.0460,0.03290,0.0491,0.00751
20,0_11,3.956624e-08,0.080538,0.027044,1,0,3.189496,-0.001863,0.944745,-0.086710,...,-0.000034,0.562611,1,0,1204.695376,5,0.1185,0.03895,0.0734,0.03944
40,0_12,2.014306e-08,0.044340,0.019405,1,0,2.179471,-0.000821,0.972760,-0.026485,...,0.000006,0.347130,2,0,1307.900000,3,0.0522,0.02336,0.0469,0.02719
60,0_13,2.400482e-08,0.056238,0.017800,1,0,2.219783,-0.001350,0.980654,-0.048117,...,-0.000014,0.271950,3,0,1365.978294,4,0.0509,0.02897,0.0327,0.02457
80,0_14,1.724124e-08,0.042506,0.024766,1,0,2.412590,-0.000219,0.966406,0.429391,...,-0.000012,2.363053,4,0,1144.787500,6,0.0912,0.07686,0.0964,0.08304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100880,9_4,2.109971e-07,0.085310,0.021940,4,0,2.649813,-0.002241,0.866363,0.056508,...,-0.000018,0.431340,3,0,1314.253231,6,0.0425,0.01345,0.0374,0.03294
100900,9_5,1.526912e-07,0.067886,0.025938,4,0,1.965845,-0.001244,0.564301,0.122511,...,0.000015,0.268158,4,0,1040.690137,2,0.0491,0.01573,0.0721,0.00931
100920,9_6,2.925304e-07,0.120796,0.043184,2,1,1.646462,0.002239,0.470154,-0.016312,...,-0.000017,0.775788,5,0,1142.515861,4,0.0357,0.01171,0.0803,0.01613
100940,9_7,1.469328e-07,0.054975,0.034891,0,1,1.963904,0.000559,0.368129,-0.018669,...,-0.000016,0.204268,17,0,1207.897028,4,0.0310,0.01410,0.0689,0.04989


In [ ]:
def pct_difference(a: int | float, b: int | float) -> np.float64:
    return np.float64(1 - abs(b - a) / a)

def compare_params(a: ParametersSet, b: ParametersSet) -> np.float64:
    return np.mean(
        [
            pct_difference(a.detection_period, b.detection_period), 
            pct_difference(a.drop_threshold, b.drop_threshold), 
            pct_difference(a.reversal_threshold, b.reversal_threshold), 
            pct_difference(a.tp_pct, b.tp_pct),
            pct_difference(a.sl_pct, b.sl_pct)
        ]
    )

In [ ]:
groups: list[list[Dataset]] = []
computed_indexes: list[int] = []
for this_index, this in enumerate(datasets):
    if this_index in computed_indexes: continue
    computed_indexes.append(this_index)
    buf: list[Dataset] = [this]
    for other_index, other in enumerate(datasets):
        if other_index in computed_indexes or this.name == other.name: continue
        score = compare_params(this.strategy_params[0], other.strategy_params[0])
        if score >= 0.85:
            computed_indexes.append(other_index)
            buf.append(other)
    groups.append(buf)

In [ ]:
dataset_to_regime: dict[str, int] = {}
for regime, group in enumerate(groups):
    for ds in group:
        dataset_to_regime[ds.name] = regime
df["regime"] = df["dataset"].map(dataset_to_regime)
df

In [ ]:
corr = df[df["regime"] == 322].corr(method="pearson")
corr = corr.drop(columns=["dataset", "trial_rank", "regime"], index=["dataset", "trial_rank", "regime"], errors="ignore")
fig, ax = plt.subplots(figsize=(12, 10))
norm = TwoSlopeNorm(vmin=-1, vcenter=-0.0, vmax=1)
cax = ax.matshow(corr.values, cmap="RdBu", norm=norm)
fig.colorbar(cax)
ax.set_xticks(np.arange(len(corr.columns)))
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.columns)
plt.tight_layout()
plt.show()

In [89]:
df[["dataset", "regime"]].to_csv(f"{root}/data/dataset_regime_map.csv", index=False)